In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
print("begin")

begin


In [13]:
path = "filter_data/clean method_is method.json"
import json
# from sklearn.model_selection import train_test_split

MIN_LENGHT = 20
def load_data(path):
    lines = open(path).readlines()
    data = [json.loads(x) for x in lines]
    data2 = [item for item in data if len(item) > MIN_LENGHT]
    return data2

def split2line(l):
    split_token = [";", "{"]
    result = []
    line = []
    for item in l:
        line.append(item)
        if item in split_token:
            result.append(line)
            line = []
    result.append(line)
    return result

data2 = load_data(path)

data3 = [split2line(item) for item in data2]



# train_data, test_data = train_test_split(data3, test_size=0.33, random_state=42)
# len(train_data), len(test_data)

def split2pair(l):
    result = []
    for i in range(1, len(l)):
        input = []
        for j in range(i):
            input += l[j]
        output = l[i]
        result.append((input.copy(),output.copy()))
    return result

# data4 = []
# for item in data3:
#     data4 += split2pair(item)

def split2pair2(l):
    result = []
    for i in range(1, len(l)):
        input = []
        for j in range(i):
            input += l[j]
            for k in range(len(l[i])):
                input += l[i][:k]
                output = l[i][k:]
                result.append((input.copy(),output.copy()))
    return result

f = open("pair_methods20000.txt", "w")


def wrap_s2p(raw_data):
    data5 = []
    pivot = 0
    for i, item in enumerate(raw_data):
        # if i % 100 == 0:
        #     print("complete{}".format(i))
        # print(data5)
        data5 += split2pair(item)
        # temp = split2pair(item)
        # for item2 in temp:
        #     # print("add", pivot)
        #     json_str = json.dumps(item2) + '\n'
        #     f.write(json_str)
        pivot += 1
        # if pivot % 100 == 0:
        #     print(pivot)
    # if pivot > 15000:
    #     break
    # print(data5[0])
    return [(" ".join(line[0]), " ".join(line[1])) for line in data5]

# train_data, test_data = wrap_s2p(train_data),  wrap_s2p(test_data)
# train_data[0]
train_data = wrap_s2p(data3)

In [17]:
import tensorflow as tf
# import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ImportError: Traceback (most recent call last):
  File "C:\Users\sy\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\sy\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\sy\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\sy\AppData\Local\Programs\Python\Python38\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\sy\AppData\Local\Programs\Python\Python38\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed while impor

In [0]:
def unicode_to_ascii(s):
    return s
    # return ''.join(c for c in unicodedata.normalize('NFD', s)
    #     if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    # print(w)
    # w = unicode_to_ascii(w.lower().strip())

    # # 在单词与跟在其后的标点符号之间插入一个空格
    # # 例如： "he is a boy." => "he is a boy ."
    # # 参考：https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    # w = re.sub(r"([?.!,¿])", r" \1 ", w)
    # w = re.sub(r'[" "]+', " ", w)

    # # 除了 (a-z, A-Z, ".", "?", "!", ",")，将所有字符替换为空格
    # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    # w = w.rstrip().strip()

    # 给句子加上开始和结束标记
    # 以便模型知道何时开始和结束预测
    w = '<start> ' + w + ' <end>'
    return w

In [0]:
def create_dataset(data, num_examples):
    inp_lang, targ_lang  = [preprocess_sentence(item[0]) for item in data[:num_examples]], [preprocess_sentence(item[1]) for item in data[:num_examples]]
    return targ_lang, inp_lang

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples = 10000000):
    data2 = load_data(path)

    data3 = [split2line(item) for item in data2]
    data = wrap_s2p(data3)
    targ_lang, inp_lang = create_dataset(data, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path, num_examples)

# 计算目标张量的最大长度 （max_length)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [10]:
# 采用 80 - 20 的比例切分训练集和验证集
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 显示长度
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

4727 4727 1182 1182


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [12]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
8 ----> <start>
34 ----> private
17 ----> builder
3 ----> (
443 ----> typename
65 ----> type
1 ----> ,
13 ----> string
36 ----> name
4 ----> )
6 ----> {
11 ----> this
2 ----> .
65 ----> type
7 ----> =
65 ----> type
5 ----> ;
9 ----> <end>

Target Language; index to word mapping
1 ----> <start>
10 ----> this
6 ----> .
23 ----> name
8 ----> =
23 ----> name
3 ----> ;
2 ----> <end>


In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 8
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset_val = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)

In [14]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([8, 397]), TensorShape([8, 119]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [16]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# 样本输入
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (8, 397, 1024)
Encoder Hidden state shape: (batch size, units) (8, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 隐藏层的形状 == （批大小，隐藏层大小）
    # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
    # 这样做是为了执行加法以计算分数  
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # 分数的形状 == （批大小，最大长度，1）
    # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
    # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
    attention_weights = tf.nn.softmax(score, axis=1)

    # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [18]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (8, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (8, 397, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # 用于注意力
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # 编码器输出 （enc_output） 的形状 == （批大小，最大长度，隐藏层大小）
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）
    x = self.embedding(x)

    # x 在拼接 （concatenation） 后的形状 == （批大小，1，嵌入维度 + 隐藏层大小）
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 将合并后的向量传送到 GRU
    output, state = self.gru(x)

    # 输出的形状 == （批大小 * 1，隐藏层大小）
    output = tf.reshape(output, (-1, output.shape[2]))

    # 输出的形状 == （批大小，vocab）
    x = self.fc(output)

    return x, state, attention_weights

In [20]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (8, 1986)


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
from nltk.translate.bleu_score import sentence_bleu
def make_sentence(seq, lang):
    input = seq[0]
    l = [lang.index_word[int(x)] for x in input if int(x) != 0]
    return " ".join(l[1:-1])

def get_bleu_result(candidate, reference):
    reference = [[item for item in reference.split(" ")]]
    candidate = [item for item in candidate.split(" ")[:-1]]
    score = sentence_bleu(reference, candidate)
    return score

def translate(it, show = False):
    input, output = it
    sentence = make_sentence(input, inp_lang)
    real_sentence = make_sentence(output, targ_lang)
    result, sentence, attention_plot = evaluate(sentence)
    bleu_score = get_bleu_result(result, real_sentence)
    if show:
        print('Input: %s' % (sentence))
        print('Real: %s' % (real_sentence))
        print('Pred: {}'.format(result))
        print("bleu: {}".format(bleu_score))
        attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
        plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return bleu_score

In [0]:
# @tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # 教师强制 - 将目标词作为下一个输入
    for t in range(1, targ.shape[1]):
      # 将编码器输出 （enc_output） 传送至解码器
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # 使用教师强制
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
import time
EPOCHS = 1

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 20 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # 每 2 个周期（epoch），保存（检查点）一次模型
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.4202
Epoch 1 Batch 20 Loss 0.2882
Epoch 1 Batch 40 Loss 0.2535
Epoch 1 Batch 60 Loss 0.3657
Epoch 1 Batch 80 Loss 0.1263
Epoch 1 Batch 100 Loss 0.2500
Epoch 1 Batch 120 Loss 0.4151
Epoch 1 Batch 140 Loss 0.1758
Epoch 1 Batch 160 Loss 0.1774
Epoch 1 Batch 180 Loss 0.1382
Epoch 1 Batch 200 Loss 0.2191
Epoch 1 Batch 220 Loss 0.1027
Epoch 1 Batch 240 Loss 0.1494
Epoch 1 Batch 280 Loss 0.1627
Epoch 1 Batch 300 Loss 0.1543
Epoch 1 Batch 320 Loss 0.1133
Epoch 1 Batch 340 Loss 0.0931
Epoch 1 Batch 360 Loss 0.1891
Epoch 1 Batch 380 Loss 0.1838
Epoch 1 Batch 400 Loss 0.2112
Epoch 1 Batch 420 Loss 0.1578


In [0]:
import numpy as np

def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # 存储注意力权重以便后面制图
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # 预测的 ID 被输送回模型
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
!pip install matplotlib 
!pip install ticker

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
iterator = iter(dataset)
input, output = next(iterator)

In [0]:
translate(next(iterator), show = True)

In [0]:
def get_average_bleu():
    result = []
    # iterator = iter(dataset_val)
    while True:
        iter = iter(dataset_val)
        result.append(translate(iter.next()))
        try:
            result.append(translate(iter.next()))
        except:
            break
    score = np.mean(result)
    print("validation bleu score",score)
    return score

get_average_bleu()

In [0]:
iter = dataset_val.as_numpy_iterator()

In [0]:
iter

In [0]:
iter.next()